In [186]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression, LinearRegression
import statsmodels.formula.api as smf
from statsmodels.formula.api import logit
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale
import sklearn.linear_model as skl_lm
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
import datetime
from datetime import date
import regressor

In [187]:
df = pd.read_csv("/Users/lilitharutyunyan/Documents/GitHub/Compas-recidivism-analysis/data-cleaned/compas-scores-two-years-clean.csv", parse_dates=["c_jail_in", "c_jail_out"])
df

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,score_text,v_type_of_assessment,v_decile_score,v_score_text,in_custody,out_custody,start,end,event,two_year_recid
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,Low,Risk of Violence,1,Low,2014-07-07,2014-07-14,0,327,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,Low,Risk of Violence,1,Low,2013-01-26,2013-02-05,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,Low,Risk of Violence,3,Low,2013-06-16,2013-06-16,0,63,0,1
3,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,High,Risk of Violence,6,Medium,NaN,NaN,0,1174,0,0
4,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,...,Low,Risk of Violence,1,Low,NaN,NaN,0,1102,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6211,10995,raheem smith,raheem,smith,2013-10-20,Male,1995-06-28,20,Less than 25,African-American,...,High,Risk of Violence,9,High,2014-04-07,2014-04-27,0,169,0,0
6212,10996,steven butler,steven,butler,2013-11-23,Male,1992-07-17,23,Less than 25,African-American,...,Medium,Risk of Violence,5,Medium,2013-11-22,2013-11-24,1,860,0,0
6213,10997,malcolm simmons,malcolm,simmons,2014-02-01,Male,1993-03-25,23,Less than 25,African-American,...,Low,Risk of Violence,5,Medium,2014-01-31,2014-02-02,1,790,0,0
6214,10999,winston gregory,winston,gregory,2014-01-14,Male,1958-10-01,57,Greater than 45,Other,...,Low,Risk of Violence,1,Low,2014-01-13,2014-01-14,0,808,0,0


In [188]:
df.columns

Index(['id', 'name', 'first', 'last', 'compas_screening_date', 'sex', 'dob',
       'age', 'age_cat', 'race', 'juv_fel_count', 'decile_score',
       'juv_misd_count', 'juv_other_count', 'priors_count',
       'days_b_screening_arrest', 'c_jail_in', 'c_jail_out', 'c_case_number',
       'c_offense_date', 'c_arrest_date', 'c_days_from_compas',
       'c_charge_degree', 'c_charge_desc', 'is_recid', 'r_case_number',
       'r_charge_degree', 'r_days_from_arrest', 'r_offense_date',
       'r_charge_desc', 'r_jail_in', 'r_jail_out', 'violent_recid',
       'is_violent_recid', 'vr_case_number', 'vr_charge_degree',
       'vr_offense_date', 'vr_charge_desc', 'type_of_assessment', 'score_text',
       'v_type_of_assessment', 'v_decile_score', 'v_score_text', 'in_custody',
       'out_custody', 'start', 'end', 'event', 'two_year_recid'],
      dtype='object')

In [189]:
#df['Date_of_Jail_In'] = pd.to_datetime(df['c_jail_in']).dt.date
#df['Time_of_Jail_In'] = pd.to_datetime(df['c_jail_in']).dt.time

In [190]:
#df['Date_of_Jail_Out'] = pd.to_datetime(df['c_jail_out']).dt.date
#df['Time_of_Jail_Out'] = pd.to_datetime(df['c_jail_out']).dt.time

In [191]:
#strftime(df[['Date_of_Jail_In']])

In [192]:
#df["age_at_arrest"]=df["Date_of_Jail_In"]-df["dob"]

In [193]:
df['days_in_jail']=df['c_jail_out']-df['c_jail_in']
df['days_in_jail'] =df['days_in_jail'].astype('category').cat.codes

In [194]:
features = pd.concat([df.age,
                    pd.get_dummies(df.sex, prefix='sex',drop_first=True),
                    pd.get_dummies(df.race, prefix='race',drop_first=True),  
                    pd.get_dummies(df.c_charge_degree, prefix='degree',drop_first=True), # felony or misdemeanor charge ('f' or 'm')
                    df.priors_count,
                    df.days_in_jail,
                    df.is_recid,
                    df.event,
                    df.end],
                    axis=1)

target = df.decile_score

In [195]:
x = features.values
y=target.values
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.33, random_state=42)

In [196]:
linearreg = LinearRegression()
linearreg.fit(X_train,y_train)
y_pred = linearreg.predict(X_test)

In [197]:
r2_score(y_test, y_pred)

0.45885281077761364

In [198]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train= sc_X.fit_transform(X_train)
X_test= sc_X.fit_transform(X_test)

In [203]:
logisticregression = LogisticRegression()
logisticregression.fit(X_train,y_train)
y_pred = linearreg.predict(X_test)
print(y_pred)

[9.1721328  5.06498528 9.55305108 ... 7.77677281 6.34247202 7.47210565]


In [204]:
r2_score(y_test, y_pred)

-1.195334390562127

In [205]:
features_new = pd.concat([df.age,
                    pd.get_dummies(df.sex, prefix='sex',drop_first=True),
                    pd.get_dummies(df.race, prefix='race',drop_first=True),  
                    pd.get_dummies(df.c_charge_degree, prefix='degree',drop_first=True), # felony or misdemeanor charge ('f' or 'm')
                    df.priors_count,
                    df.days_in_jail,
                    df.is_recid,
                    df.event,
                    df.end,
                    df.decile_score],
                    axis=1)

target_new = df.two_year_recid

In [206]:
x_new = features_new.values
y_new=target_new.values
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(x_new,y_new, test_size=0.33, random_state=42)

In [207]:
linearreg = LinearRegression()
linearreg.fit(X_train_new,y_train_new)
y_pred_new = linearreg.predict(X_test_new)

In [208]:
r2_score(y_test_new, y_pred_new)

0.869954064709805

In [184]:
# optimal parameter

from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

param_grid = {'alpha':np.arange(0.1,1,0.05)}

lasso = Lasso()

lasso_cv = GridSearchCV(lasso, param_grid, cv = 5)

lasso_cv.fit(X_train_new, y_train_new)
print(lasso_cv.best_params_)

print('Training Score:', lasso_cv.score(X_train_new, y_train_new))
print('Test Score:', lasso_cv.score(X_test_new, y_test_new))

{'alpha': 0.1}
Training Score: 0.5499450649494468
Test Score: 0.5608608793641896


In [211]:
lasso = Lasso(alpha=0.1, normalize=True)
lasso.fit(X_train_new, y_train_new)
lasso_pred = lasso.predict(X_test_new)
#Returns the coefficient of determination R^2 of the prediction.
lasso.score(X_test_new, y_test_new)
r2_score(y_test_new, lasso_pred)

-1.1569401601629892e-05

In [212]:
###smth wrong with the code, doesn't change

In [213]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing

degree=5
polyreg_scaled=make_pipeline(PolynomialFeatures(degree),LinearRegression())
polyreg_scaled.fit(X_train_new,y_train_new)

Pipeline(steps=[('polynomialfeatures', PolynomialFeatures(degree=5)),
                ('linearregression', LinearRegression())])

In [215]:
y_pred_new = linearreg.predict(X_test_new)
r2_score(y_test_new, y_pred_new)

0.869954064709805

# Logistic regression is a more appropriate technique (than standard multiple linear regression) for estimating the partial and complete associations among a set of predictors and a dichotomous outcome variable.
 *Employing a logistic transformation assures that the predicted values do not fall outside the bounds of the outcome variable (in this case “0” and “1”). 

In [173]:
features_log = pd.concat([df.age,
                    pd.get_dummies(df.sex, prefix='sex',drop_first=True),
                    pd.get_dummies(df.race, prefix='race',drop_first=True),  
                    pd.get_dummies(df.c_charge_degree, prefix='degree',drop_first=True), # felony or misdemeanor charge ('f' or 'm')
                    df.priors_count,
                    df.days_in_jail,
                    df.is_recid,
                    df.event,
                    df.end,
                    df.decile_score
                    ],
                    axis=1)

target_log = df.two_year_recid

In [176]:
x_log = features_log.values
y_log=target_log.values
X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(x_log,y_log, test_size=0.33, random_state=42)

In [182]:
from sklearn.linear_model import LogisticRegression
logisticregression = LogisticRegression()
logisticregression.fit(X_train_log,y_train_log)
proba = logisticregression.predict_proba(X_test_log)
print(proba)

[[7.07491951e-03 9.92925080e-01]
 [9.99980708e-01 1.92918035e-05]
 [2.12654742e-02 9.78734526e-01]
 ...
 [5.60793398e-03 9.94392066e-01]
 [9.99995207e-01 4.79254013e-06]
 [3.73510201e-01 6.26489799e-01]]


/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [183]:
r2_score(y_test_log, y_pred_log)

0.869954064709805